## Resumo

### Parâmetros

Será usado um kernel `rbf` em um kernel SVC. O `C` será otimizado pelo Optuna a fim de obter um resultado melhor. O `gamma` foi deixado no valor padrão que é `scale`, a fórmula referente a esse valor está disponível na documentação do scikit-learn.

### Representação adotada

A representação adotada foi escalar os valores númericos com um scaler do sklearn (no momento, um RobustScaler). Criar uma bag of words a partir do resumo. Criar uma bag of items com o elenco. Extrair o ano da data de estreia. E, por fim, descartar a história original e o titulo por não serem tão relevantes para os resultados. A coluna "adulo" foi deixada mesmo estando distribuida de maneira extremamente desuniforme, assumindo-se que o modelo irá incorporá-la melhor caso ela seja distribuída mais uniformemente.

Porém, o resultado mais efeitvo usa apenas a bag of words e a bag of items, descartando os demais fatores. Isso provavelmente está relacionado com a grande ausência de informações (NaNs) dos demais parâmetros.

### Método adotado

O método adotado foi um classificador SVC com um kernel `rbf`, já que sua acurácia se mostrou maior que o `linear`. O kernel `poly` não foi testado por falta de tempo, mas espera-se que ele seja menos preciso em grais baixos ou consideravelmente mais lento em grais altos. 

Os resultados do método Random Forest são lvemente inferiores, mas conseguem incorporar mais informações para a predição.

In [2]:
import pandas as pd
import numpy as np
from competicao_am.gerar_resultado_teste import gerar_saida_teste

In [3]:
df_treino = pd.read_csv('datasets/movies_amostra.csv')
df_predict = pd.read_csv('datasets/movies_amostra_teste_ex.csv')

df_treino.head()

,id,titulo,adulto,orcamento,idioma_original,popularidade,data_de_estreia,resumo,receita,duracao,genero,ator_1,ator_2,ator_3,ator_4,ator_5,dirigido_por,escrito_por_1,escrito_por_2,historia_original
0,86295,Treasure of the Yankee Zephyr,False,0,en,2.0,1981-11-28,In a lake high in the mountains of New Zealand...,0.0,108.0,Action,Ken Wahl,Lesley Ann Warren,Donald Pleasence,George Peppard,Bruno Lawrence,David Hemmings,Everett De Roche,NaN,NaN
1,289198,Redeemer,False,0,es,2.0,2014-09-18,A former hit-man for a drug cartel becomes a v...,0.0,88.0,Action,Marko Zaror,Loreto Aravena,Mauricio Diocares,José Luís Mósca,Noah Segan,Ernesto Díaz Espinoza,NaN,NaN,NaN
2,24382,Big Deal on Madonna Street,False,0,it,11.0,1958-06-06,"Peppe, formerly a boxer, organizes the break-i...",0.0,106.0,Comedy,Vittorio Gassman,Renato Salvatori,Memmo Carotenuto,Rossana Rory,Carla Gravina,Mario Monicelli,Agenore Incrocci,Furio Scarpelli,NaN
3,479,Shaft,False,46000000,en,13.0,2000-06-15,New York police detective John Shaft arrests W...,107196498.0,99.0,Action,Samuel L. Jackson,Jeffrey Wright,Christian Bale,Busta Rhymes,Dan Hedaya,John Singleton,NaN,NaN,NaN
4,37712,Pistol Opera,False,0,ja,2.0,2001-10-27,"As one of Suzuki's last fims, it is related to...",0.0,112.0,Action,Makiko Esumi,Sayoko Yamaguchi,Hanae Kan,Masatoshi Nagase,Mikijiro Hira,Seijun Suzuki,Kazunori Ito,NaN,NaN


In [4]:
from competicao_am.preprocessamento_atributos_competicao import DataframePreprocessing
from IPython.display import display

df_amostra = df_treino.sample(100, random_state=7)

preprocessor = DataframePreprocessing(df_amostra, df_amostra, 'genero')

dfs = preprocessor.generate_dataframes()

for k, v in dfs.items():
    display(v[0].head())

0/100
0/100


,adulto,orcamento,popularidade,receita,duracao
1306,False,609000.0,1.272727,3202000.0,0.235294
2037,False,0.0,-0.060606,0.0,0.000000
568,False,0.0,-0.181818,0.0,1.470588
1897,False,0.0,-0.181818,0.0,-0.647059
2498,False,0.0,0.060606,0.0,-5.352941


,index,adulto,orcamento,popularidade,receita,duracao,index,boi_Roscoe 'Fatty' Arbuckle,boi_Boris Quercia,boi_Chuck Norris,boi_Scott Caan,boi_Matheus Souza
0,1306,False,609000.0,1.272727,3202000.0,0.235294,0,0,0,0,0,0
1,2037,False,0.0,-0.060606,0.0,0.000000,1,0,0,0,0,0
2,568,False,0.0,-0.181818,0.0,1.470588,2,0,0,0,0,0
3,1897,False,0.0,-0.181818,0.0,-0.647059,3,0,0,0,0,0
4,2498,False,0.0,0.060606,0.0,-5.352941,4,0,0,0,0,0


,index,adulto,orcamento,popularidade,receita,duracao,index,resumo_000,resumo_11,resumo_119,...,resumo_york,resumo_yoshiyuki,resumo_you,resumo_young,resumo_yuan,resumo_yuanlong,resumo_zatoichi,resumo_zealand,resumo_zoe,resumo_zócalo
0,1306,False,609000.0,1.272727,3202000.0,0.235294,0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2037,False,0.0,-0.060606,0.0,0.000000,1,0.0,0.134747,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,568,False,0.0,-0.181818,0.0,1.470588,2,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1897,False,0.0,-0.181818,0.0,-0.647059,3,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2498,False,0.0,0.060606,0.0,-5.352941,4,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


,boi_Roscoe 'Fatty' Arbuckle,boi_Boris Quercia,boi_Chuck Norris,boi_Scott Caan,boi_Matheus Souza
0,0,0,0,0,0
1,0,0,0,0,0
2,0,0,0,0,0
3,0,0,0,0,0
4,0,0,0,0,0


,resumo_000,resumo_11,resumo_119,resumo_15,resumo_18,resumo_1913,resumo_1938,resumo_2013,resumo_20th,resumo_252,...,resumo_york,resumo_yoshiyuki,resumo_you,resumo_young,resumo_yuan,resumo_yuanlong,resumo_zatoichi,resumo_zealand,resumo_zoe,resumo_zócalo
0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.134747,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


,index,adulto,orcamento,popularidade,receita,duracao,index,boi_Roscoe 'Fatty' Arbuckle,boi_Boris Quercia,boi_Chuck Norris,...,resumo_york,resumo_yoshiyuki,resumo_you,resumo_young,resumo_yuan,resumo_yuanlong,resumo_zatoichi,resumo_zealand,resumo_zoe,resumo_zócalo
0,1306,False,609000.0,1.272727,3202000.0,0.235294,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2037,False,0.0,-0.060606,0.0,0.000000,1,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,568,False,0.0,-0.181818,0.0,1.470588,2,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1897,False,0.0,-0.181818,0.0,-0.647059,3,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2498,False,0.0,0.060606,0.0,-5.352941,4,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## Otimização dos parâmetros

In [5]:
import optuna
import pickle # Serialização dos resultados

In [6]:
import matplotlib.pyplot as plt
from sklearn import svm

from base_am.resultado import Fold
from base_am.avaliacao import Experimento
import competicao_am.metodo_competicao as mc
import competicao_am.avaliacao_competicao as ac

## Um pouco sobre SVC's

### Kernel

O `kernel` indica o tipo de "hiper-plano" usado para separar as classes.

Os kernels a serem considerados são: `linear`, `rbf` e `poly`

#### `poly`: degree

Grau do polinômio a ser usado. Quanto maior o valor, maior o tempo gasto para o treino.

### gamma

Parâmetro usado para "hiper-planos" não lineares. Quanto maior, mais ele tenta en encaixar o modelo perfeitamente.

Valores muito altos caminhão em direção de um _overfitting_.

### C

C é o parâmetro de penalidade do erro. Ele controla o prejuízo entre paredes de decisões suaves e classificar os pontos corretamente.



Links úteis:

- [A guide to SVM parameter tuning](https://towardsdatascience.com/a-guide-to-svm-parameter-tuning-8bfe6b8a452c)

In [7]:
def experimento(metodo: mc.MetodoCompeticao, num_trials: int = 2, otimizador:ac.OtimizacaoObjetivo = ac.OtimizacaoObjetivoSVCRbf, ml_method = None) -> Experimento:
    folds = Fold.gerar_k_folds(df_treino, val_k=5, col_classe="genero",
                                num_repeticoes=1, num_folds_validacao=4, num_repeticoes_validacao=1)

    ClasseObjetivo = ac.WrapperOtimizacaoObjetivo(otimizador, metodo=metodo)

    experimento = Experimento(folds, ml_method=ml_method, ClasseObjetivoOtimizacao=ClasseObjetivo, num_trials=num_trials)

    return experimento

In [7]:

teste_exp = experimento(mc.MetodoCompeticaoValidacao_A)
teste_exp.macro_f1_avg

0/1800
1000/1800
0/600


In [8]:
va_exp = experimento(mc.MetodoCompeticaoValidacao_A, 10)
print(va_exp.macro_f1_avg)

0/1800
1000/1800
0/600
0/1800
1000/1800
0/600
0/1800
1000/1800
0/600
0/1800
1000/1800
0/600
[I 2020-08-19 23:37:02,190] Finished trial#0 with value: 0.7321453360563505 with parameters: {'exp_cost': 2.9343616008230637}. Best is trial#0 with value: 0.7321453360563505.
0/1800
1000/1800
0/600
0/1800
1000/1800
0/600
0/1800
1000/1800
0/600
0/1800
1000/1800
0/600
[I 2020-08-19 23:41:07,987] Finished trial#1 with value: 0.7321453360563505 with parameters: {'exp_cost': 4.796536502777316}. Best is trial#0 with value: 0.7321453360563505.
0/1800
1000/1800
0/600
0/1800
1000/1800
0/600
0/1800
1000/1800
0/600
0/1800
1000/1800
0/600
[I 2020-08-19 23:45:14,821] Finished trial#2 with value: 0.7321453360563505 with parameters: {'exp_cost': 1.431165748120622}. Best is trial#0 with value: 0.7321453360563505.
0/1800
1000/1800
0/600
0/1800
1000/1800
0/600
0/1800
1000/1800
0/600
0/1800
1000/1800
0/600
[I 2020-08-19 23:49:21,187] Finished trial#3 with value: 0.7321453360563505 with parameters: {'exp_cost': 6.1

KeyboardInterrupt: 

In [11]:
vb_exp = experimento(mc.MetodoCompeticaoValidacao_B, 10)
print(vb_exp.macro_f1_avg)

0/1800
1000/1800
0/600
0/1800
1000/1800
0/600
0/1800
1000/1800
0/600
0/1800
1000/1800
0/600
[I 2020-08-20 01:37:35,010] Finished trial#0 with value: 0.4365691982575012 with parameters: {'exp_cost': 6.922027622345463}. Best is trial#0 with value: 0.4365691982575012.
0/1800
1000/1800
0/600
0/1800
1000/1800
0/600
0/1800
1000/1800
0/600
0/1800
1000/1800
0/600
[I 2020-08-20 01:40:50,107] Finished trial#1 with value: 0.4318471745754282 with parameters: {'exp_cost': 5.237159580658876}. Best is trial#0 with value: 0.4365691982575012.
0/1800
1000/1800
0/600


KeyboardInterrupt: 

In [12]:
vc_exp = experimento(mc.MetodoCompeticaoValidacao_C, 10)
print(vc_exp.macro_f1_avg)

0/1800
1000/1800
0/600
0/1800
1000/1800
0/600
0/1800
1000/1800
0/600
0/1800
1000/1800
0/600
[I 2020-08-20 01:44:52,943] Finished trial#0 with value: 0.4342888516066461 with parameters: {'exp_cost': 5.524955299160419}. Best is trial#0 with value: 0.4342888516066461.
0/1800
1000/1800
0/600
0/1800
1000/1800
0/600


KeyboardInterrupt: 

In [14]:
vd_exp = experimento(mc.MetodoCompeticaoValidacao_D, 10)
print(vd_exp.macro_f1_avg)

0/1800
1000/1800
0/600
0/1800
1000/1800
0/600
0/1800
1000/1800
0/600
0/1800
1000/1800
0/600
[I 2020-08-20 01:49:44,883] Finished trial#0 with value: 0.42913346921340406 with parameters: {'exp_cost': 3.135254683231336}. Best is trial#0 with value: 0.42913346921340406.
0/1800
1000/1800
0/600
0/1800
1000/1800
0/600
0/1800
1000/1800
0/600
0/1800
1000/1800
0/600
[I 2020-08-20 01:53:05,962] Finished trial#1 with value: 0.4352841434003213 with parameters: {'exp_cost': 6.360168521651669}. Best is trial#1 with value: 0.4352841434003213.
0/1800
1000/1800
0/600
0/1800
1000/1800
0/600
0/1800
1000/1800
0/600
0/1800
1000/1800
0/600
[I 2020-08-20 01:56:28,318] Finished trial#2 with value: 0.43392209863355236 with parameters: {'exp_cost': 2.0552990386157566}. Best is trial#1 with value: 0.4352841434003213.
0/1800
1000/1800
0/600
0/1800
1000/1800
0/600
0/1800
1000/1800
0/600
0/1800
1000/1800
0/600
[I 2020-08-20 01:59:47,503] Finished trial#3 with value: 0.43392209863355236 with parameters: {'exp_cost':

KeyboardInterrupt: 

In [8]:
va_exp2 = experimento(mc.MetodoCompeticaoValidacao_A, 10, otimizador=ac.OtimizacaoObjetivoRandomForest)
print(va_exp2.macro_f1_avg)

0/1800
1000/1800
0/600
0/1800
1000/1800
0/600
0/1800
1000/1800
0/600
0/1800
1000/1800
0/600
[I 2020-08-20 02:03:55,807] Finished trial#0 with value: 0.6119459998383177 with parameters: {'min_samples_split': 0.208511002351287, 'max_features': 0.36016224672107905, 'num_arvores': 2}. Best is trial#0 with value: 0.6119459998383177.
0/1800
1000/1800
0/600
0/1800
1000/1800
0/600
0/1800
1000/1800
0/600
0/1800
1000/1800
0/600
[I 2020-08-20 02:04:12,214] Finished trial#1 with value: 0.5870072991431701 with parameters: {'min_samples_split': 0.15116628631591988, 'max_features': 0.07337794540855652, 'num_arvores': 1}. Best is trial#0 with value: 0.6119459998383177.
0/1800
1000/1800
0/600
0/1800
1000/1800
0/600
0/1800
1000/1800
0/600
0/1800
1000/1800
0/600
[I 2020-08-20 02:04:44,681] Finished trial#2 with value: 0.6731526249993971 with parameters: {'min_samples_split': 0.19829036364801306, 'max_features': 0.1939553705810037, 'num_arvores': 5}. Best is trial#2 with value: 0.6731526249993971.
0/1800


KeyboardInterrupt: 

In [9]:
vb_exp2 = experimento(mc.MetodoCompeticaoValidacao_B, 10, otimizador=ac.OtimizacaoObjetivoRandomForest)
print(vb_exp2.macro_f1_avg)

0/1800
1000/1800
0/600
0/1800
1000/1800
0/600
0/1800
1000/1800
0/600
0/1800
1000/1800
0/600
[I 2020-08-20 02:14:26,417] Finished trial#0 with value: 0.6239094939815613 with parameters: {'min_samples_split': 0.29327752025099646, 'max_features': 0.45170095764394175, 'num_arvores': 1}. Best is trial#0 with value: 0.6239094939815613.
0/1800
1000/1800
0/600
0/1800
1000/1800
0/600
0/1800
1000/1800
0/600
0/1800
1000/1800
0/600
[I 2020-08-20 02:14:56,924] Finished trial#1 with value: 0.6765245954037751 with parameters: {'min_samples_split': 0.2239900907102091, 'max_features': 0.4460679360009405, 'num_arvores': 5}. Best is trial#1 with value: 0.6765245954037751.
0/1800
1000/1800
0/600
0/1800
1000/1800
0/600
0/1800
1000/1800
0/600
0/1800
1000/1800
0/600
[I 2020-08-20 02:15:20,188] Finished trial#2 with value: 0.6194333392755584 with parameters: {'min_samples_split': 0.26921234713999254, 'max_features': 0.32614944103177107, 'num_arvores': 2}. Best is trial#1 with value: 0.6765245954037751.
0/1800

KeyboardInterrupt: 

In [10]:
vc_exp2 = experimento(mc.MetodoCompeticaoValidacao_C, 10, otimizador=ac.OtimizacaoObjetivoRandomForest)
print(vc_exp2.macro_f1_avg)

0/1800
1000/1800
0/600
0/1800
1000/1800
0/600
0/1800
1000/1800
0/600
0/1800
1000/1800
0/600
[I 2020-08-20 02:25:22,372] Finished trial#0 with value: 0.6361771524459957 with parameters: {'min_samples_split': 0.19393032203208588, 'max_features': 0.43177092727971433, 'num_arvores': 1}. Best is trial#0 with value: 0.6361771524459957.
0/1800
1000/1800
0/600
0/1800
1000/1800
0/600
0/1800
1000/1800
0/600
0/1800
1000/1800
0/600
[I 2020-08-20 02:25:49,509] Finished trial#1 with value: 0.663744756050479 with parameters: {'min_samples_split': 0.21390631629616108, 'max_features': 0.21419120258262464, 'num_arvores': 5}. Best is trial#1 with value: 0.663744756050479.
0/1800
1000/1800
0/600
0/1800
1000/1800
0/600
0/1800
1000/1800
0/600
0/1800
1000/1800
0/600
[I 2020-08-20 02:26:22,895] Finished trial#2 with value: 0.6953479072203521 with parameters: {'min_samples_split': 0.07666923895978722, 'max_features': 0.46859516220000336, 'num_arvores': 4}. Best is trial#2 with value: 0.6953479072203521.
0/1800

KeyboardInterrupt: 

In [ ]:
vd_exp2 = experimento(mc.MetodoCompeticaoValidacao_D, 10, otimizador=ac.OtimizacaoObjetivoRandomForest)
print(vd_exp2.macro_f1_avg)